In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import tensorflow as tf
from tensorflow import keras 
import numpy as np
import pandas as pd

In [2]:
poems_data = open("/content/drive/MyDrive/poems/poems.txt", "r").read()

In [3]:
data_corpus = poems_data.lower().split("\n")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_corpus)
total_words = len(tokenizer.word_index) + 1
print(data_corpus)
print(tokenizer.word_index)
print(total_words)

['i wandered lonely as a cloud', "that floats on high o'er vales and hills,", 'when all at once i saw a crowd,', 'a host, of golden daffodils;', 'beside the lake, beneath the trees,', 'fluttering and dancing in the breeze.', 'continuous as the stars that shine', 'and twinkle on the milky way,', 'they stretched in never-ending line', 'along the margin of a bay:', 'ten thousand saw i at a glance,', 'tossing their heads in sprightly dance.', 'the waves beside them danced; but they', 'out-did the sparkling waves in glee:', 'a poet could not but be gay,', 'in such a jocund company:', 'i gazed—and gazed—but little thought', 'what wealth the show to me had brought:', 'for oft, when on my couch i lie', 'in vacant or in pensive mood,', 'they flash upon that inward eye', 'which is the bliss of solitude;', 'and then my heart with pleasure fills,', 'and dances with the daffodils.', 'whose woods these are i think i know.   ', 'his house is in the village though;   ', 'he will not see me stopping he

In [4]:
# create input sequences using list of tokens
input_seqs = []
for line in data_corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_seqs = token_list[:i+1]
		input_seqs.append(n_gram_seqs)

# pad sequences 
max_seqs_len = max([len(x) for x in input_seqs])
input_seqs = np.array(pad_sequences(input_seqs, maxlen=max_seqs_len, padding='pre'))

# create predictors and label
preds, labels = input_seqs[:,:-1],input_seqs[:,-1]

labels = ku.to_categorical(labels, num_classes=total_words)

In [5]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_seqs_len - 1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer= 'RMSprop',
    metrics=['accuracy']
)

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 22, 100)           238000    
                                                                 
 bidirectional (Bidirectiona  (None, 22, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1190)              120190    
                                                                 
 dense_1 (Dense)             (None, 2380)              2834580   
                                                        

In [6]:
import os
checkpoint_path = "/content/drive/MyDrive/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [7]:
import os
checkpoint_path = "/content/drive/MyDrive/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.load_weights(checkpoint_path)

In [ ]:
model_history = model.fit(preds, labels, epochs=250, verbose=1, callbacks=[cp_callback])

In [ ]:
import matplotlib.pyplot as plt
model_acc = model_history.history['accuracy']
model_loss = model_history.history['loss']


nepochs = range(len(model_acc))

plt.plot(nepochs, model_acc, 'b', label='Training accuracy')
plt.title('Training accuracy')
plt.ylabel("Accuracy")
plt.xlabel("Epochs")

plt.figure()

plt.plot(nepochs, model_loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()

plt.show()

In [8]:
start_txt = "the weather is nice"
nwords = 110
  
for _ in range(nwords):
	token_list = tokenizer.texts_to_sequences([start_txt])[0]
	token_list = pad_sequences([token_list], maxlen=max_seqs_len-1, padding='pre')
	predicted_word = np.argmax(model.predict(token_list, verbose=0))
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted_word:
			output_word = word
			break
	start_txt += " " + output_word

start_txt = start_txt.split()
n = 6
[' '.join(start_txt[i:i+n]) for i in range(0,len(start_txt),n)]

['the weather is nice glee done',
 'mirth out out like furnace for',
 'fine plashless was no mouth up',
 'with her fearful storm have come',
 'was ever gloom their comet down',
 'the snow to fatal song enough',
 "an england's worm her door is",
 'year or road over night i',
 'those for this forests and her',
 'mouth and like what a thoughts',
 'in the hour everything cottage grieve',
 'happen mildly simple the bay lash',
 'the a a thoughts of the',
 'a is forget a is she',
 'sound’s the road up was those',
 'who once it dreams in he',
 'know love anyway throw their hair',
 'lives express rooms climb on your',
 'work and shot who see my']